In [ ]:
class MeanSquaredError:
    def compute(self, target, prediction):
        esum = 0
        delta = np.subtract(target,prediction)
        for i in delta:
            esum += i ** 2
        return esum/len(delta)    
    def differentiate(self, prediction, target):
        return np.multiply(np.subtract(target, prediction), 2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import math

In [ ]:
iris = pd.read_csv("/content/drive/MyDrive/minor/python/Iris.csv")
iris = iris.sample(frac=1).reset_index(drop=True) # Shuffle the data
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,10,4.9,3.1,1.5,0.1,Iris-setosa
1,19,5.7,3.8,1.7,0.3,Iris-setosa
2,110,7.2,3.6,6.1,2.5,Iris-virginica
3,135,6.1,2.6,5.6,1.4,Iris-virginica
4,27,5.0,3.4,1.6,0.4,Iris-setosa


In [ ]:
X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
# to numpy array 
X = np.array(X)
X[:5]

array([[4.9, 3.1, 1.5, 0.1],
       [5.7, 3.8, 1.7, 0.3],
       [7.2, 3.6, 6.1, 2.5],
       [6.1, 2.6, 5.6, 1.4],
       [5. , 3.4, 1.6, 0.4]])

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)

# One hot encode the species 
y = iris.Species
y = one_hot_encoder.fit_transform(np.array(y).reshape(-1, 1))
y[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [ ]:
# Split data up in train/test data and then into train/validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
def NeuralNetwork(X_train, y_train, X_val=None, y_val=None, epochs=10, nodes=[], lr=0.15):
    hidden_layers = len(nodes) - 1
    weights = init_weights(nodes) #weigths initialiseren

    for epoch in range(1, epochs + 1):
      # Model trainen
        weights = train(X_train, y_train, lr, weights)

        if(epoch % 20 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(accuracy(X_train, y_train, weights)))
            if X_val.any():
                print("Validation Accuracy:{}".format(accuracy(X_val, y_val, weights)))
            
    return weights

In [ ]:
def init_weights(nodes):
    # Initialize weights with random values in [-1, 1] ook de bias
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)]
              for j in range(nodes[i])]
        weights.append(np.matrix(w))
    
    return weights

In [ ]:
def forwardprop(x, weights, layers):
  # de weight inputs 
    activations, layer_input = [x], x
    for j in range(layers):
        activation = sig(np.dot(layer_input, weights[j].T))
        activations.append(activation)
        layer_input = np.append(1, activation) # Augment with bias
    
    return activations

In [ ]:
def backprop(y, activations, weights, layers):
    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal) # Error at output
    
    for j in range(layers, 0, -1):
        current_activation = activations[j]
        
        if(j > 1):
            # Augment previous activation
            previous_activation = np.append(1, activations[j-1])
        else:
            # First hidden layer, previous ctivation is input without bias
            previous_activation = activations[0]
        
        delta = np.multiply(error, dsig(current_activation))
        weights[j-1] += lr * np.multiply(delta.T, previous_activation)

        w = np.delete(weights[j-1], [0], axis=1) # Remove bias from weights
        error = np.dot(delta, w) # Calculate error for current layer
    
    return weights

In [ ]:
def train(X, Y, lr, weights):
    layers = len(weights)
    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Augment feature vector
        
        #first go trouhg outputs forward
        activations = forwardprop(x, weights, layers)
        # then back propagation for weight updating
        weights = backprop(y, activations, weights, layers)

    return weights

In [ ]:
# Sigmoid of x 
def sig(x):
  return 1 / (1 + np.exp(-x))

# Derivative of sigmoid of x
def dsig(x):
    return np.multiply(x, 1-x)

#Relu of x
def relu( x):
    return x if x > 0 else 0 

#Derivative relu of x
def drelu( x):
    return 1.0 if x > 0 else 0 

In [ ]:
def predict(item, weights):
    layers = len(weights)
    item = np.append(1, item) # Augment feature vector
    
    # forward propagation
    activations = forwardprop(item, weights, layers)
    
    outputFinal = activations[-1].A1
    index = find_max_activation(outputFinal)

    # Initialize prediction vector to zeros
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1  # set prediciton to 1
    
    # return prediction vector
    return y 

def find_max_activation(output):
    # Find max activation in output
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

In [ ]:
def accuracy(X, Y, weights):
    corr_pred = 0
    # For every prediction
    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        guess = predict(x, weights)

        # Add to corr_pred if prediction is correct
        if(y == guess):
            corr_pred += 1

    return corr_pred / len(X)

In [ ]:
features = len(X[0]) # Number of features
output = len(y[0]) # Number of outputs / classes

layers = [features, 5, 10, output] # Number of nodes in layers
lr, epochs = 0.15, 100

weights = NeuralNetwork(X_train, y_train, X_val, y_val, epochs=epochs, nodes=layers, lr=lr);

Epoch 20
Training Accuracy:0.6666666666666666
Validation Accuracy:0.6153846153846154
Epoch 40
Training Accuracy:0.9824561403508771
Validation Accuracy:0.9230769230769231
Epoch 60
Training Accuracy:0.8245614035087719
Validation Accuracy:1.0
Epoch 80
Training Accuracy:0.9649122807017544
Validation Accuracy:0.9230769230769231
Epoch 100
Training Accuracy:0.7894736842105263
Validation Accuracy:1.0
